In [47]:
from replay_reader import read_snoopy, parse_bullets, width, height, player_r, bullet_r

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [177]:
def feats(row, player):
    pos = np.array([row[f'x{player}'] / width, row[f'y{player}'] / height])
    pos_score = (1 - 4 * (pos[0] - 0.5) ** 2 +
                 np.log(2 / 3 - abs(pos[1] - 2 / 3) + 0.01))
    
    bullets = parse_bullets(row['bullets'])
    if bullets.empty:
        bullet_score = 0
    else:
        bullets['x'] /= width
        bullets['y'] /= height
        leeway = (player_r + bullet_r) / max(width, height)
        bullet_score = sum(np.log(np.linalg.norm(vec - pos) - leeway) for vec in bullets.query(f'player != "Player {player}"')[['x', 'y']].values)
    
    curr_angle = np.arctan2(row[f'y{3-player}'] - row[f'y{player}'],
                            row[f'x{3-player}'] - row[f'x{player}'])
    orientation_score = np.pi * 2 - abs(row[f'a{player}'] - curr_angle)
    
    return (pos_score, bullet_score, orientation_score)

In [178]:
def feat_df(game_df):
    rows = []
    for row in game_df.to_dict(orient='records'):
        pos, bullets, angle = feats(row, 1)
        pos2, bullets2, angle2 = feats(row, 2)
        rows.append({
            'pos1': pos,
            'bullet1': bullets,
            'orient1': angle,
            'pos2': pos2,
            'bullet2': bullets2,
            'orient2': angle2
        })
    return pd.DataFrame(rows)

In [214]:
class QLearner:
    def __init__(self, pos=0.0, bullets=0.0, angle=0.0, courage=1.0):
        self.pos = pos
        self.bullets = bullets
        self.angle = angle
        self.courage = courage
        
    def learn(self, f_df, winner, alpha=0.01, gamma=0.9):
        final_time = max(f_df.index)
        s = {}
        for index in f_df[~f_df.isna().any(axis=1)].index:
            for i in (1, 2):
                s[i] = np.sum(np.array([self.pos, self.bullets, self.angle]) * f_df.loc[index, [f'pos{i}', f'bullet{i}', f'orient{i}']].values)
            for i in (1, 2):
                real_q = gamma ** (final_time - index) * (1 if winner == i else -1)
                q = self.courage * s[i] - s[3 - i]
                print(real_q, q, self)
                delta = real_q - q
                # courage is really just adding new features with the same weights: it's a hyperparameter, not a parameter
                self.pos += alpha * delta * self.courage * f_df.loc[index, f'pos{i}']
                self.pos -= alpha * delta * f_df.loc[index, f'pos{3-i}']
                self.bullets += alpha * delta * self.courage * f_df.loc[index, f'bullet{i}']
                self.bullets -= alpha * delta * f_df.loc[index, f'bullet{3-i}']
                self.angle += alpha * delta * self.courage * f_df.loc[index, f'orient{i}']
                self.angle -= alpha * delta * f_df.loc[index, f'orient{3-i}']
                
        return (self.pos, self.bullets, self.angle)
    
    def __repr__(self):
        return f"QLearner(pos={self.pos:.2}, bullets={self.bullets:.2}, angle={self.angle:.2}, courage={self.courage:.2})"

In [215]:
q = QLearner()
for n_round, i in tqdm(enumerate(range(1, 41))):
    print(f"Game {i}")
    g = read_snoopy(f'../replays/game{i}.snoopy')
    if g.result != 0:
        winner = 1 if g.result == 1 else -1
        q.learn(feat_df(g.to_df()), winner, alpha=1/(10 + n_round))
        print(q)

0it [00:00, ?it/s]

Game 1


1it [00:00,  2.73it/s]

0.0011790184577738603 0.0 QLearner(pos=0.0, bullets=0.0, angle=0.0, courage=1.0)
-0.0011790184577738603 0.0 QLearner(pos=7.1e-05, bullets=0.0, angle=-0.00028, courage=1.0)
0.0013100205086376223 0.0014032911934899735 QLearner(pos=0.00014, bullets=0.0, angle=-0.00056, courage=1.0)
-0.0013100205086376223 -0.0014032911934899735 QLearner(pos=0.00014, bullets=0.0, angle=-0.00054, courage=1.0)
0.0014555783429306916 0.0012895691893319349 QLearner(pos=0.00013, bullets=0.0, angle=-0.00051, courage=1.0)
-0.0014555783429306916 -0.0012895691893319349 QLearner(pos=0.00014, bullets=0.0, angle=-0.00055, courage=1.0)
0.0016173092699229906 0.0014807961171382413 QLearner(pos=0.00015, bullets=0.0, angle=-0.00059, courage=1.0)
-0.0016173092699229906 -0.0014807961171382413 QLearner(pos=0.00016, bullets=0.0, angle=-0.00062, courage=1.0)
0.001797010299914434 0.0016336425403777409 QLearner(pos=0.00017, bullets=0.0, angle=-0.00066, courage=1.0)
-0.001797010299914434 -0.0016336425403777409 QLearner(pos=0.00018, 

2it [00:00,  2.72it/s]

...

25it [00:06,  4.15it/s]

-0.020275559590445275 6.336115667445258e+179 QLearner(pos=1.8e+179, bullets=9e+180, angle=6.1e+179, courage=1.0)
-0.020275559590445275 -6.336115667445258e+179 QLearner(pos=1.6e+179, bullets=9e+180, angle=6e+179, courage=1.0)
-0.022528399544939195 5.7584600289341e+179 QLearner(pos=1.4e+179, bullets=9e+180, angle=5.8e+179, courage=1.0)
-0.022528399544939195 -5.7584600289341e+179 QLearner(pos=1.2e+179, bullets=9e+180, angle=5.7e+179, courage=1.0)
-0.025031555049932437 5.234620527946241e+179 QLearner(pos=1.1e+179, bullets=9e+180, angle=5.6e+179, courage=1.0)
-0.025031555049932437 -5.234620527946241e+179 QLearner(pos=9e+178, bullets=9e+180, angle=5.4e+179, courage=1.0)
-0.027812838944369374 4.755640915838669e+179 QLearner(pos=7.4e+178, bullets=9e+180, angle=5.3e+179, courage=1.0)
-0.027812838944369374 -4.755640915838669e+179 QLearner(pos=6e+178, bullets=9e+180, angle=5.2e+179, courage=1.0)
-0.030903154382632636 4.317310192685765e+179 QLearner(pos=4.6e+178, bullets=9e+180, angle=5.1e+179, co

26it [00:06,  4.17it/s]


-0.06461081889226677 -2.4818901511117323e+180 QLearner(pos=3.1e+180, bullets=3e+180, angle=-1.7e+180, courage=1.0)
-0.0717897987691853 2.3291787828963755e+180 QLearner(pos=3e+180, bullets=3e+180, angle=-1.7e+180, courage=1.0)
-0.0717897987691853 -2.3291787828963755e+180 QLearner(pos=3e+180, bullets=3e+180, angle=-1.8e+180, courage=1.0)
-0.07976644307687256 2.1809970026849606e+180 QLearner(pos=2.9e+180, bullets=3e+180, angle=-1.8e+180, courage=1.0)
-0.07976644307687256 -2.1809970026849606e+180 QLearner(pos=2.8e+180, bullets=3e+180, angle=-1.9e+180, courage=1.0)
-0.08862938119652507 2.0287785085669565e+180 QLearner(pos=2.7e+180, bullets=3e+180, angle=-1.9e+180, courage=1.0)
-0.08862938119652507 -2.0287785085669565e+180 QLearner(pos=2.7e+180, bullets=3e+180, angle=-1.9e+180, courage=1.0)
-0.09847709021836118 1.8737399376895443e+180 QLearner(pos=2.6e+180, bullets=3e+180, angle=-2e+180, courage=1.0)
-0.09847709021836118 -1.8737399376895443e+180 QLearner(pos=2.5e+180, bullets=3e+180, angle=

KeyboardInterrupt: 